In [4]:
import numpy as np
import MDAnalysis as mda
import nmrformd

In [5]:
import sys, os, git
current_path = os.getcwd()
git_repo = git.Repo(current_path, search_parent_directories=True)
git_path = git_repo.git.rev_parse("--show-toplevel")

In [6]:
def save_result(data, name, aniso = False):
    """Save the correlation functions in dictionary"""
    if not os.path.exists("raw_data/"):
        os.makedirs("raw_data/")
    saving_file = "raw_data/" + name + ".npy"
    t = data.t
    f = data.f
    if aniso:
        C1 = data.gij[0]
        C2 = data.gij[1]
        C3 = data.gij[2]
    else:
        C = data.gij[0]
    R1 = data.R1
    R2 = data.R2
    N = data.group_j.atoms.n_atoms
    try:
        previous_dictionary = np.load(saving_file, allow_pickle=True)
        t_prev = np.real(previous_dictionary.item()["t"])
        assert len(t_prev) == len(t)
        if aniso:
            C1_prev = np.real(previous_dictionary.item()["C1"])
            C2_prev = np.real(previous_dictionary.item()["C2"])
            C3_prev = np.real(previous_dictionary.item()["C3"])
        else:
            C_prev = np.real(previous_dictionary.item()["C"])
        R1_prev = np.real(previous_dictionary.item()["R1"])
        R2_prev = np.real(previous_dictionary.item()["R2"])
        N_prev = np.real(previous_dictionary.item()["N"])
        if aniso:
            C1 = (C1*N + C1_prev*N_prev) / (N_prev + N)
            C2 = (C2*N + C2_prev*N_prev) / (N_prev + N)
            C3 = (C3*N + C3_prev*N_prev) / (N_prev + N)
        else:
            C = (C*N + C_prev*N_prev) / (N_prev + N)
        R1 = (R1*N + R1_prev*N_prev) / (N_prev + N)
        R2 = (R2*N + R2_prev*N_prev) / (N_prev + N)
        N += N_prev
    except:
        pass
    if aniso:
        dictionary = {
        "t": t,
        "f": f,
        "C1": C1,
        "C2": C2,
        "C3": C3,
        "N": N,
        "R1": R1,
        "R2": R2,
        }
    else:
        dictionary = {
        "t": t,
        "f": f,
        "C": C,
        "N": N,
        "R1": R1,
        "R2": R2,
        }
    np.save(saving_file, dictionary)
    return N

In [8]:
while 1<2:   
    # high res
    N = "N4000_HR"
    for mystep in [1, 2, 4, 8, 16, 32]:
        datapath = git_path + "/data/bulk-water-high-dumping-rate/bulk-water/raw-data/N4000_HR/"
        xtc = [datapath+"prod1.xtc", datapath+"prod2.xtc",
               datapath+"prod3.xtc", datapath+"prod4.xtc",
               datapath+"prod5.xtc", datapath+"prod6.xtc",
               datapath+"prod7.xtc", datapath+"prod8.xtc",
               datapath+"prod9.xtc", datapath+"prod10.xtc"]
        u = mda.Universe(datapath+"prod1.tpr", xtc)
        u.transfer_to_memory(step = mystep)
        dt = np.round(u.trajectory.dt,2)
        hydrogen = u.select_atoms("name HW1 HW2")
        n_hydrogen = hydrogen.n_atoms
        intra = nmrformd.NMR(u, atom_group = hydrogen, neighbor_group = hydrogen, type_analysis = "intra_molecular", number_i=5)
        inter = nmrformd.NMR(u, atom_group = hydrogen, neighbor_group = hydrogen, type_analysis = "inter_molecular", number_i=1)
        n_intra = save_result(intra, name = "N4000_HR_intra_dt" + str(dt))
        n_inter = save_result(inter, name = "N4000_HR_inter_dt" + str(dt))
        print(n_hydrogen, mystep, n_intra, dt)

8000 1 137 0.2
8000 1 138 0.2


KeyboardInterrupt: 